In [6]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

event_prefix = '../dataset/tr.csv'
event_prefix = '../dataset/2020_100_sorted.csv'

input_for_trainning = '../dataset/aux/tr-aux.csv'

def put_each_hit_in_a_single_line_train(event_prefix):

    auxdf3  = pd.DataFrame()
    totfinaldfaux = pd.DataFrame()

    totdf = pd.read_csv(event_prefix) #,skiprows=0,nrows=1)
    print(totdf.shape)

    columnsPerHit=8
    maximumHitPerTrack=20
    positionB=1
    #positionE=40 #17+8+8+8+8
    positionE=49 #17+8+8+8+8
    amount_of_hits = maximumHitPerTrack*columnsPerHit

    for k in range(3,4):
    #for k in range(3,10):
        print("K: ", k)
        print("interval: ", positionB,positionE)
        

        totdf3 = totdf.iloc[:,positionB:positionE]
        print("totdf3.shape: ", totdf3.shape)
        print("totdf3: ", totdf3)
        print(totdf3.columns.shape[0])
        
        #rename column names to append in resulta dataframe
        for i in range(totdf3.columns.shape[0]):
            totdf3 = totdf3.rename(columns={totdf3.columns[i]: i})

        totfinaldfaux = totfinaldfaux.append(totdf3)

        positionB=positionB+columnsPerHit
        positionE=positionE+columnsPerHit

    print(totfinaldfaux.shape)

    return totfinaldfaux

#ds = put_each_hit_in_a_single_line_train(event_prefix)


In [5]:
def prepare_training_data(path, cilyndrical=False ):
    
    df = pd.read_csv(path, header=0, engine='python')
    
    start_hits = 9
    interval = 11
    df = df.iloc[:, start_hits:]
    
    # x, y, z coordinates
    if cilyndrical == False:
        bp=1
        ep=4
        bpC=10
        epC=11
       
    # cilyndrical coordinates    
    elif cilyndrical == True:
        bp=4
        ep=7
        bpC=10
        epC=11     

    df_hits_positions = df.iloc[:, np.r_[bp:ep,
                            bp+(interval*1):ep+(interval*1),
                            bp+(interval*2):ep+(interval*2),
                            bp+(interval*3):ep+(interval*3)]]


    # data values of particles
    df_hits_values = df.iloc[:, np.r_[bpC:epC,
                                   bpC+(interval*1):epC+(interval*1),
                                   bpC+(interval*2):epC+(interval*2),
                                   bpC+(interval*3):epC+(interval*3)]]

    frame = [df_hits_positions, df_hits_values]
    X =  pd.concat(frame, axis=1)
    X = X.iloc[:,:]
    y = df.iloc[:, np.r_[bp+(interval*4):(bp+(interval*4)+3)]]
        
    return X, y

def dataprep(file):
    #Data preparation

    #load track file
    df = pd.read_csv(file,)

    df1 = df.iloc[:,10:54]

    #Separate values and features for LSTM
    dataX2 = df1.iloc[:, [ 0,1,2,8,9,10,16,17,18,24,25,26]]
    dataXfeatures = df1.iloc[:, [ 6,14,22,30 ]]

    b = dataX2.values.flatten()
    bfeat = dataXfeatures.values.flatten()
    n_patterns = len(df)
    
    X     = dataX2
    Xfeat = dataXfeatures

    #XMLP= df1.iloc[:, [ 2,3,4,9,10,11,12,17,18,19,20,24]]

    resorg= df1.iloc[:, [ 32,33,34]]
    #resorgfile=diraux+'/resorg'
    resorg.to_csv('resorg.csv', index = False)

    #prepare data to be approximated to inference data
    df_all_hits = df1.iloc[:, [ 31,32,33,34,35,36,37,38]]
    yy=df_all_hits.iloc[:,:]
    columns = yy.columns
    yclone = np.copy(yy)
    #dfyclone = pd.DataFrame.from_records(yclone)
    
    dfyclone = pd.DataFrame(yclone, columns=columns)

    return X, Xfeat, dfyclone, resorg

In [13]:
x, x_, y, y_ = dataprep('../dataset/if.csv')
y

,hit_id_4,x_4,y_4,z_4,volume_id_4,layer_id_4,module_id_4,value_4
0,67364.0,250.080002,75.234100,-96.000000,13.0,2.0,382.0,3.0
1,67305.0,239.695999,-97.476601,-72.400002,13.0,2.0,378.0,1.0
2,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,80852.0,496.027008,-66.991798,187.399994,13.0,6.0,974.0,3.0
4,67322.0,261.901001,-2.233630,-76.000000,13.0,2.0,380.0,2.0
5,67830.0,251.514999,-83.622002,49.799999,13.0,2.0,418.0,3.0
6,67358.0,251.589005,57.645401,-71.199997,13.0,2.0,382.0,1.0
7,67849.0,256.264008,-56.144600,-1.800000,13.0,2.0,419.0,1.0
8,67825.0,253.684006,-80.025398,-36.599998,13.0,2.0,418.0,2.0
9,68365.0,259.143005,40.400101,101.800003,13.0,2.0,461.0,4.0


In [24]:
prefix = '../dataset/2020_100_sorted.csv'  
df = pd.read_csv(prefix, header=0, engine='python')


hit = 19
begin_hit = 'hit_id_'
end_hit = 'value_'
begin = begin_hit+str(hit)
end = end_hit+str(hit)

df = df.loc[:, begin:end]
df

,hit_id_19,x_19,y_19,z_19,rho_19,eta_19,phi_19,volume_id_19,layer_id_19,module_id_19,value_19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
event_prefix = '../dataset/2020_100_sorted.csv'
#event_prefix = '../dataset/tr.csv'

X, y = prepare_training_data(event_prefix, cilyndrical=False)
X.head()

print('[Data] X.shape', X.shape)
print('[Data] y.shape', y.shape)

X.head()


[Data] X.shape (40, 16)
[Data] y.shape (40, 3)


,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,value_0,value_1,value_2,value_3
0,-24.585400,-20.413200,-13.975500,-56.510399,-45.227600,-31.276600,-92.247803,-70.983597,-50.016201,-138.167007,-101.019997,-73.405899,0.319292,0.331103,0.321971,0.344224
1,-14.056000,-28.675501,-21.582600,-31.304100,-64.726898,-48.192902,-49.917801,-104.850998,-77.674202,-72.214203,-154.794998,-114.222000,0.366232,0.361090,0.354962,0.317347
2,-27.423901,-17.235300,-64.558403,-61.020302,-37.752800,-142.643005,-98.834396,-60.154701,-229.826004,-147.701996,-88.006104,-341.398010,0.687946,0.655072,0.651616,0.666651
3,-24.421700,-20.596399,-23.423300,-55.571999,-46.090599,-52.555500,-89.466301,-72.973297,-83.822403,-134.436996,-107.330002,-124.806999,0.381427,0.393663,0.394846,0.374068
4,-12.655300,29.405399,-60.221001,-26.612101,65.975998,-133.429993,-40.456200,107.658997,-215.979996,-55.553200,162.884003,-323.472992,0.619523,0.638750,0.652402,0.654377


In [7]:
size_x = X.shape[0]
print('size_x ', size_x)

time_steps = 4
num_features =4 

X_tmp = X.values.flatten()
X_new = np.reshape(X_tmp,(size_x,time_steps, num_features))
X_new

size_x  40


array([[[-2.45853996e+01, -2.04132004e+01, -1.39755001e+01,
         -5.65103989e+01],
        [-4.52276001e+01, -3.12765999e+01, -9.22478027e+01,
         -7.09835968e+01],
        [-5.00162010e+01, -1.38167007e+02, -1.01019997e+02,
         -7.34058990e+01],
        [ 3.19292307e-01,  3.31102908e-01,  3.21970522e-01,
          3.44223738e-01]],

       [[-1.40559998e+01, -2.86755009e+01, -2.15825996e+01,
         -3.13041000e+01],
        [-6.47268982e+01, -4.81929016e+01, -4.99178009e+01,
         -1.04850998e+02],
        [-7.76742020e+01, -7.22142029e+01, -1.54794998e+02,
         -1.14222000e+02],
        [ 3.66231680e-01,  3.61090362e-01,  3.54961574e-01,
          3.17346901e-01]],

       [[-2.74239006e+01, -1.72353001e+01, -6.45584030e+01,
         -6.10203018e+01],
        [-3.77528000e+01, -1.42643005e+02, -9.88343964e+01,
         -6.01547012e+01],
        [-2.29826004e+02, -1.47701996e+02, -8.80061035e+01,
         -3.41398010e+02],
        [ 6.87946081e-01,  6.55071557e-

In [10]:
event_prefix = '../dataset/2020_100_sorted.csv'
#df = pd.read_csv(event_prefix, header=1, index_col=['hit_id_0'], engine='python')
df = pd.read_csv(event_prefix, header=0, engine='python')

#df = pd.read_csv(event_prefix)
df = df.iloc[:, 9:]
df.head()

,hit_id_0,x_0,y_0,z_0,rho_0,eta_0,phi_0,volume_id_0,layer_id_0,module_id_0,...,x_19,y_19,z_19,rho_19,eta_19,phi_19,volume_id_19,layer_id_19,module_id_19,value_19
0,17420.0,-24.585400,-20.413200,-13.975500,31.955292,-0.708570,-2.448648,8.0,2.0,98.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17502.0,-14.056000,-28.675501,-21.582600,31.935177,-0.427043,-2.026553,8.0,2.0,99.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17472.0,-27.423901,-17.235300,-64.558403,32.390213,-0.768700,-2.580497,8.0,2.0,98.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17457.0,-24.421700,-20.596399,-23.423300,31.947317,-0.704657,-2.440963,8.0,2.0,98.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18882.0,-12.655300,29.405399,-60.221001,32.013031,-0.385684,1.977209,8.0,2.0,109.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
bp=1
ep=4

bpC=10
epC=11

interval=11

df_hits = df.iloc[:, np.r_[bp:ep, bpC:epC,
                        bp+(interval*1):ep+(interval*1), bpC+(interval*1):epC+(interval*1),
                        bp+(interval*2):ep+(interval*2),  bpC+(interval*2):epC+(interval*2),
                        bp+(interval*3):ep+(interval*3),  bpC+(interval*3):epC+(interval*3)]
                 ]

df_hits.head()



,x_0,y_0,z_0,value_0,x_1,y_1,z_1,value_1,x_2,y_2,z_2,value_2,x_3,y_3,z_3,value_3
0,-24.585400,-20.413200,-13.975500,0.319292,-56.510399,-45.227600,-31.276600,0.331103,-92.247803,-70.983597,-50.016201,0.321971,-138.167007,-101.019997,-73.405899,0.344224
1,-14.056000,-28.675501,-21.582600,0.366232,-31.304100,-64.726898,-48.192902,0.361090,-49.917801,-104.850998,-77.674202,0.354962,-72.214203,-154.794998,-114.222000,0.317347
2,-27.423901,-17.235300,-64.558403,0.687946,-61.020302,-37.752800,-142.643005,0.655072,-98.834396,-60.154701,-229.826004,0.651616,-147.701996,-88.006104,-341.398010,0.666651
3,-24.421700,-20.596399,-23.423300,0.381427,-55.571999,-46.090599,-52.555500,0.393663,-89.466301,-72.973297,-83.822403,0.394846,-134.436996,-107.330002,-124.806999,0.374068
4,-12.655300,29.405399,-60.221001,0.619523,-26.612101,65.975998,-133.429993,0.638750,-40.456200,107.658997,-215.979996,0.652402,-55.553200,162.884003,-323.472992,0.654377


In [125]:

# data values of particles
df_values = df.iloc[:, np.r_[bpC:epC,
                               bpC+(interval*1):epC+(interval*1),
                               bpC+(interval*2):epC+(interval*2),
                               bpC+(interval*3):epC+(interval*3)]]

df_values.head()


,value_0,value_1,value_2,value_3
0,0.319292,0.331103,0.321971,0.344224
1,0.366232,0.361090,0.354962,0.317347
2,0.687946,0.655072,0.651616,0.666651
3,0.381427,0.393663,0.394846,0.374068
4,0.619523,0.638750,0.652402,0.654377


In [126]:
y = df.iloc[:, np.r_[bp+(interval*4):(bp+(interval*4)+3)]]
y.head()


,x_4,y_4,z_4
0,-216.294006,-144.802002,-111.400002
1,-107.084999,-237.365005,-174.199997
2,-223.113007,-128.664001,-511.399994
3,-205.817001,-158.632996,-188.600006
4,-71.434700,250.406006,-491.000000


In [247]:
event_prefix = '../dataset/tr.csv'
#df = pd.read_csv(event_prefix, header=1, index_col=['hit_id_0'], engine='python')
df2 = pd.read_csv(event_prefix, header=0, engine='python')

#df = pd.read_csv(event_prefix)
df2 = df2.iloc[:, 9:]
df2.head()

,hit_id_0,x_0,y_0,z_0,volume_id_0,layer_id_0,module_id_0,value_0,hit_id_1,x_1,...,module_id_3,value_3,hit_id_4,x_4,y_4,z_4,volume_id_4,layer_id_4,module_id_4,value_4
0,16198.0,28.176500,16.392799,-36.742001,8.0,2.0,106.0,0.424473,22039.0,61.622898,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,16172.0,24.454599,20.559601,-46.045200,8.0,2.0,106.0,0.441045,22049.0,54.083000,...,283.0,0.471207,57668.0,186.074005,184.733002,-321.000000,13.0,2.0,305.0,4.0
2,17892.0,27.172100,16.773899,47.709900,8.0,2.0,122.0,0.586853,24389.0,61.629002,...,904.0,0.628501,60384.0,235.876007,123.170998,459.000000,13.0,2.0,583.0,3.0
3,15796.0,27.274500,-16.963400,-27.858801,8.0,2.0,103.0,0.354827,22680.0,62.152699,...,423.0,0.367249,57946.0,225.315994,-133.841003,-170.600006,13.0,2.0,337.0,2.0
4,15882.0,28.062901,-15.327300,-24.451200,8.0,2.0,103.0,0.317890,22677.0,63.334702,...,423.0,0.335427,58384.0,222.526001,-132.548996,-142.000000,13.0,2.0,377.0,1.0


In [277]:
bp=1
ep=4
bpC=7
epC=8

interval=11

dataX2=df2.iloc[:, np.r_[bp:ep,
                        bp+(interval*1):ep+(interval*1),
                        bp+(interval*2):ep+(interval*2),
                        bp+(interval*3):ep+(interval*3)]]

dataXfeatures=df2.iloc[:, np.r_[bpC:epC,
                               bpC+(interval*1):epC+(interval*1),
                               bpC+(interval*2):epC+(interval*2),
                               bpC+(interval*3):epC+(interval*3)]]

#evalX=(bp+(interval*3))
#evalX3=evalX+3
#print(bp+(interval*4),(bp+(interval*4)+3))

y=df2.iloc[:, np.r_[bp+(interval*4):(bp+(interval*4)+3)]]

#dataX3.head()

b = dataX2.values.flatten()
bfeat=dataXfeatures.values.flatten()
n_patterns=len(df2)
X     = np.reshape(b,(n_patterns,4,3))
#Xfeat = np.reshape(bfeat,(n_patterns,3,4))
Xfeat = np.reshape(bfeat,(n_patterns,4,1))


IndexError: positional indexers are out-of-bounds

In [250]:
#dataXfeatures.head()
dataX2.head()


,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3
0,28.176500,16.392799,-36.742001,61.622898,36.088402,-71.028000,99.855797,58.901199,-110.329002,0.000000,0.000000,0.000000
1,24.454599,20.559601,-46.045200,54.083000,46.787300,-93.296402,86.075302,76.923897,-145.781006,125.985001,117.313004,-213.507996
2,27.172100,16.773899,47.709900,61.629002,36.919399,117.415001,100.313004,58.340401,194.701004,150.225998,83.974800,293.032013
3,27.274500,-16.963400,-27.858801,62.152699,-38.337200,-53.205799,98.817902,-60.440701,-79.715401,147.410995,-89.166702,-114.681000
4,28.062901,-15.327300,-24.451200,63.334702,-35.107899,-45.408298,100.864998,-56.908401,-67.908096,148.787994,-85.756302,-96.906502


In [119]:
y.head()

,x_4,y_4,z_4
0,0.000000,0.000000,0.000000
1,186.074005,184.733002,-321.000000
2,235.876007,123.170998,459.000000
3,225.315994,-133.841003,-170.600006
4,222.526001,-132.548996,-142.000000
